In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clusterconda inconda install -c conda-forge geopystall -c conda-forge geopying stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!conda install -c conda-forge html5lib--yes
print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /home/yahyaabualhaj/anaconda3

  added / updated specs:
    - geopy


The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/yahyaabualhaj/anaconda3

  added / updated specs:
    - folium=0.5.0


The following NEW packages will be INSTALLED:

  altair             conda-forge/linux-64::altair-3.2.0-py37_0
  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             conda-forge/noarch::folium-0.5.0-py_0
  vincent            conda-forge/noarch::vincent-0.4.4-py_1


Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [3]:
conda install -c conda-forge html5lib 

Solving environment: done

## Package Plan ##

  environment location: /home/yahyaabualhaj/anaconda3

  added / updated specs:
    - html5lib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    html5lib-1.0.1             |             py_0          89 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          89 KB

The following packages will be SUPERSEDED by a higher-priority channel:

  html5lib           pkgs/main/linux-64::html5lib-1.0.1-py~ --> conda-forge/noarch::html5lib-1.0.1-py_0



html5lib-1.0.1       | 89 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


1 . To import the table (Scrape) from  Wikipedia page and build Dataframe.
 

In [31]:
df = pd.read_html("https://gpsarab.com/shop11/en/content/12-zip-code-in-jordan")[0]
print(f'The number of (rows,cols) for [df] : {df.shape} ')

The number of (rows,cols) for [df] : (458, 5) 


In [32]:
df['City'] = df['City'].apply(lambda x: x.split("|")[0].strip() ).astype(str) 
df['Location'] = df['Location'].apply(lambda x: x.split("|")[0].strip()).astype(str) 
df.head()

,City,Location,Zip Code,Latitude,Longitude
0,Amman,Abdoun Al Janobi (S),11183,31.942011,35.881741
1,Amman,Abdoun Alshamali (N),11183,31.948469,35.893509
2,Amman,Abu Alanda,11592,31.905396,35.960555
3,Amman,Abu Alya,11946,32.001043,35.970014
4,Amman,Abu-Nsair,11937,32.052860,35.876480


2. Ignore cells with a borough that is Not assigned.

In [35]:
df1 = df[df['City']=='Amman']
df1.head()

,City,Location,Zip Code,Latitude,Longitude
0,Amman,Abdoun Al Janobi (S),11183,31.942011,35.881741
1,Amman,Abdoun Alshamali (N),11183,31.948469,35.893509
2,Amman,Abu Alanda,11592,31.905396,35.960555
3,Amman,Abu Alya,11946,32.001043,35.970014
4,Amman,Abu-Nsair,11937,32.052860,35.876480



To get the geographical coordinates of Torrento

In [36]:
address = 'Amman'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Manhattan are 31.9515694, 35.9239625.



Create a map of Toronto with neighborhoods superimposed on top

In [73]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], 
                                           df1['Longitude'],
                                           df1['City'],
                                           df1['Location']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version.

In [42]:
CLIENT_ID = '5T2GGWMMZXTOSAYESR0ZQHUT0PPPY31ZGXMZBIKNCDBALKR3' # your Foursquare ID
CLIENT_SECRET = 'FDPDARQ4AZD3XAV235TPLHGGGUKUMLQOUAMTFOJVIXDOHSAM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: 5T2GGWMMZXTOSAYESR0ZQHUT0PPPY31ZGXMZBIKNCDBALKR3
CLIENT_SECRET:FDPDARQ4AZD3XAV235TPLHGGGUKUMLQOUAMTFOJVIXDOHSAM



x

In [43]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df1.loc[1,'Latitude'], 
    df1.loc[1,'Longitude'], 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5T2GGWMMZXTOSAYESR0ZQHUT0PPPY31ZGXMZBIKNCDBALKR3&client_secret=FDPDARQ4AZD3XAV235TPLHGGGUKUMLQOUAMTFOJVIXDOHSAM&v=20180605&ll=31.948469,35.893509&radius=500&limit=100'

In [44]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7d3c38b9961d002be7b8a7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Amman',
  'headerFullLocation': 'Amman',
  'headerLocationGranularity': 'city',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 31.952969004500005,
    'lng': 35.898802427879744},
   'sw': {'lat': 31.943968995499993, 'lng': 35.88821557212026}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4be1b0c6a145c928f53b4e80',
       'name': 'Gerard (جيرارد)',
       'location': {'address': 'Abdoun Circle',
        'lat': 31.948392378042097,
        'lng': 35.89228282897579,
        'labeledLatLngs': [{'label': 'display',
          'lat': 31.948392378042097,
          '


create a function to find venues for each Neighbourhood.

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=' \
              '{}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
amman_venues = getNearbyVenues(names=df1['Location'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )



Abdoun Al Janobi (S)
Abdoun Alshamali (N)
Abu Alanda
Abu Alya
Abu-Nsair
Ain Ghazal
Al Abdali
Jabal Al Akhdar
D. Al Ameerah Alia
Al Bnyat Al Shamali
Al Dalelah
Al Hashmi Al Janoubi
Al Hashmi Al Shamali
Al Hay Al Sharqi
Al Homar
Al Horreyya
Al Husnyah
Al Hussein Medical City
Al Jandweal
Al Jizah
Al Kolyah Aleslamyah
Al Kursi
Al Lubban
Al Madenah Altebyah
Al Madinah Al Riyadiah
Al Madinah Al Riyadiah
Al Mahattah
Al Mansour
Al Matabah
Al Mohajrean
Al Mowaqar
Al Muhajireen
Al Mushaqqar
Al Muwaqqar
Al Ouroubah
Al qastal
Al Qonaytrah
Al Qweismeh
Al Rabaheya
Al Rahmaniah
Al Raqim
Al Rawabi
Al Rawdah
Al Rawnaq
Al Rewaq
Al Rjoum
Al Salahyah
Al Shaheed Al Janobi
Al Shaheed Al Shamali
Al Tajamouat Industrial City
Al Yadudah
Al Yarmouk
Al Zaafaran
Al. Mansourah
Alal
Albasalah
Albashayer
Albnyat Aljanobi
Aldeyar
Alfaroq
Alforosyah
Alforqan
Alia Housing
Alkmalyah
Alkoam Algharbi
Alkoam Alsharqi
Alktaifah
Almahbah
Alqasbat
AlQusur Al Malakiah
Alwehdat
Alyasmeen
Alzaytonah
Amman City Center
Amman Mall


Let's check the size of the resulting dataframe

In [49]:
print(amman_venues.shape)
amman_venues.head()

(1276, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abdoun Al Janobi (S),31.942011,35.881741,Adam,31.941975,35.881291,Restaurant
1,Abdoun Al Janobi (S),31.942011,35.881741,Buffalo Wings & Rings,31.941047,35.884596,Wings Joint
2,Abdoun Al Janobi (S),31.942011,35.881741,Fitness First Platinum,31.941527,35.884685,Gym
3,Abdoun Al Janobi (S),31.942011,35.881741,Halim Zaman (حليم زمان),31.940512,35.882930,Café
4,Abdoun Al Janobi (S),31.942011,35.881741,Dahab Restaurant,31.941745,35.881992,Restaurant


In [52]:
amman_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abdoun Al Janobi (S),46,46,46,46,46,46
Abdoun Alshamali (N),42,42,42,42,42,42
Abu Alanda,6,6,6,6,6,6
Abu Alya,3,3,3,3,3,3
Abu-Nsair,5,5,5,5,5,5
Al Abdali,13,13,13,13,13,13
Al Bnyat Al Shamali,2,2,2,2,2,2
Al Hashmi Al Janoubi,19,19,19,19,19,19
Al Hashmi Al Shamali,4,4,4,4,4,4



Find out how many unique categories can be curated from all the returned venues

In [53]:
print('There are {} uniques categories.'.format(len(amman_venues['Venue Category'].unique())))

There are 163 uniques categories.



Analyze Each Neighborhood.

In [55]:
# one hot encoding
amman_onehot = pd.get_dummies(amman_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
amman_onehot['Neighborhood'] = amman_venues['Neighborhood'] 

fixed_columns = [col for col in amman_onehot.columns if col != 'Neighborhood' ]
fixed_columns =  ['Neighborhood'] + fixed_columns        
amman_onehot = amman_onehot[fixed_columns]

amman_onehot.head()


,Neighborhood,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Bath House,Beach,Bed & Breakfast,Beer Garden,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Business Service,Café,Campground,Candy Store,Cheese Shop,Chinese Restaurant,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Currency Exchange,Dance Studio,Department Store,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Duty-free Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lebanese Restaurant,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Nightclub,Other Great Outdoors,Other Nightlife,Paintball Field,Park,Pastry Shop,Pedestrian Plaza,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Resort,Restaurant,Road,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Street Fair,Supermarket,Sushi Restaurant,Tea Room,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Tunnel,Turkish Restaurant,Video Game Store,Video Store,Wings Joint,Women's Store,Zoo Exhibit
0,Abdoun Al Janobi (S),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Abdoun Al Janobi (S),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Abdoun Al Janobi (S),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Abdoun Al Janobi (S),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Abdoun Al Janobi (S),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Grouping rows by neighborhood.

In [56]:
amman_onehot.shape

(1276, 163)

In [57]:
amman_grouped = amman_onehot.groupby('Neighborhood').mean().reset_index()
amman_grouped

,Neighborhood,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Bath House,Beach,Bed & Breakfast,Beer Garden,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Business Service,Café,Campground,Candy Store,Cheese Shop,Chinese Restaurant,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Currency Exchange,Dance Studio,Department Store,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Duty-free Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,Gift Shop,Go Kart Track,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lebanese Restaurant,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Nightclub,Other Great Outdoors,Other Nightlife,Paintball Field,Park,Pastry Shop,Pedestrian Plaza,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Resort,Restaurant,Road,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Street Fair,Supermarket,Sushi Restaurant,Tea Room,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Tunnel,Turkish Restaurant,Video Game Store,Video Store,Wings Joint,Women's Store,Zoo Exhibit
0,Abdoun Al Janobi (S),0.000000,0.021739,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.021739,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.0,0.00000,0.000000,0.065217,0.0,0.0,0.000000,0.195652,0.000000,0.000000,0.000000,0.043478,0.00,0.000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.021739,0.000000,0.0,0.000000,0.000000,0.000000,0.021739,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.021739,0.043478,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.021739,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.043478,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.021739,0.000000,0.000000,0.00,0.043478,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.0,0.0,0.000000,0.108696,0.000000,0.021739,0.021739,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.00,0.0000,0.021739,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.0,0.000000
1,Abdoun Alshamali (N),0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.00,0.000000,0.0,0.000000,0.000000,0.023810,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.047619,0.0,0.0,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.023810,0.023810,0.000000,0


Print each neighborhood


In [58]:
num_top_venues = 5

for hood in amman_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = amman_grouped[amman_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abdoun Al Janobi (S)----
                venue  freq
0                Café  0.20
1          Restaurant  0.11
2        Burger Joint  0.07
3         Coffee Shop  0.04
4  Chinese Restaurant  0.04


----Abdoun Alshamali (N)----
                  venue  freq
0            Restaurant  0.14
1                  Café  0.14
2  Fast Food Restaurant  0.10
3        Ice Cream Shop  0.07
4                Lounge  0.05


----Abu Alanda----
                     venue  freq
0                 Pharmacy  0.17
1              Cheese Shop  0.17
2  Fruit & Vegetable Store  0.17
3         Department Store  0.17
4              Snack Place  0.17


----Abu Alya----
              venue  freq
0       Coffee Shop  0.33
1  Asian Restaurant  0.33
2        Restaurant  0.33
3  Pedestrian Plaza  0.00
4       Pastry Shop  0.00


----Abu-Nsair----
                  venue  freq
0           Pizza Place   0.4
1  Gym / Fitness Center   0.2
2         Shopping Mall   0.2
3     Food & Drink Shop   0.2
4      Airport Terminal   0.

                venue  freq
0          Skate Park  0.25
1  Falafel Restaurant  0.25
2       Grocery Store  0.25
3        Soccer Field  0.25
4    Airport Terminal  0.00


----Alforosyah----
              venue  freq
0              Farm  0.25
1          Mountain  0.25
2       Supermarket  0.25
3   Nature Preserve  0.25
4  Airport Terminal  0.00


----Alia Housing----
                       venue  freq
0                   Pharmacy  0.25
1                        Gym  0.25
2  Middle Eastern Restaurant  0.25
3                     Bakery  0.25
4           Airport Terminal  0.00


----Alkmalyah----
          venue  freq
0          Farm  0.25
1      Mountain  0.25
2   Supermarket  0.25
3  Intersection  0.25
4   Music Venue  0.00


----Almahbah----
                  venue  freq
0           Pizza Place  0.11
1         Shopping Mall  0.11
2           Coffee Shop  0.11
3  Gym / Fitness Center  0.11
4                Bistro  0.11


----Alqasbat----
                       venue  freq
0                

                venue  freq
0               Hotel  0.33
1      Scenic Lookout  0.33
2    Airport Terminal  0.00
3         Music Venue  0.00
4  Mexican Restaurant  0.00


----Jabal AlHussein----
                       venue  freq
0  Middle Eastern Restaurant  0.33
1                     Lounge  0.33
2                      Diner  0.33
3           Airport Terminal  0.00
4                Men's Store  0.00


----Jabal Amman----
                venue  freq
0              Lounge  0.07
1                 Pub  0.05
2                 Bar  0.05
3               Hotel  0.05
4  Italian Restaurant  0.05


----Jabel Al Webdeh----
                venue  freq
0                Café  0.19
1                 Bar  0.10
2  Falafel Restaurant  0.10
3               Hotel  0.10
4           Hotel Bar  0.05


----Jbeiha----
                       venue  freq
0                       Café  0.25
1                 Restaurant  0.22
2  Middle Eastern Restaurant  0.08
3       Fast Food Restaurant  0.08
4                Piz


Write a function to sort the venues in descending order.

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Create the new dataframe and display the top 10 venues for each neighborhood.

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = amman_grouped['Neighborhood']

for ind in np.arange(amman_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(amman_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abdoun Al Janobi (S),Café,Restaurant,Burger Joint,Park,Ice Cream Shop,Middle Eastern Restaurant,Chinese Restaurant,Coffee Shop,Wings Joint,Gym
1,Abdoun Alshamali (N),Restaurant,Café,Fast Food Restaurant,Ice Cream Shop,Burger Joint,Lounge,Middle Eastern Restaurant,Plaza,Pool,Pub
2,Abu Alanda,Pharmacy,Plaza,Cheese Shop,Fruit & Vegetable Store,Snack Place,Department Store,Zoo Exhibit,Dance Studio,Donut Shop,Doner Restaurant
3,Abu Alya,Coffee Shop,Restaurant,Asian Restaurant,Electronics Store,Duty-free Shop,Donut Shop,Doner Restaurant,Dog Run,Diner,Dessert Shop
4,Abu-Nsair,Pizza Place,Gym / Fitness Center,Shopping Mall,Food & Drink Shop,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Diner,Dessert Shop



Cluster Neighborhoods.

In [66]:
# set number of clusters
kclusters = 5

amman_grouped_clustering = amman_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(amman_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([4, 4, 4, 4, 4, 4, 0, 4, 4, 0], dtype=int32)

In [81]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
amman_data = df1
amman_merged = amman_data.drop('Zip Code',1)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
amman_merged = amman_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Location')
amman_merged = amman_merged.drop([16])
amman_merged.dropna(inplace=True) # check the last columns!
amman_merged.head()

,City,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amman,Abdoun Al Janobi (S),31.942011,35.881741,4.0,Café,Restaurant,Burger Joint,Park,Ice Cream Shop,Middle Eastern Restaurant,Chinese Restaurant,Coffee Shop,Wings Joint,Gym
1,Amman,Abdoun Alshamali (N),31.948469,35.893509,4.0,Restaurant,Café,Fast Food Restaurant,Ice Cream Shop,Burger Joint,Lounge,Middle Eastern Restaurant,Plaza,Pool,Pub
2,Amman,Abu Alanda,31.905396,35.960555,4.0,Pharmacy,Plaza,Cheese Shop,Fruit & Vegetable Store,Snack Place,Department Store,Zoo Exhibit,Dance Studio,Donut Shop,Doner Restaurant
3,Amman,Abu Alya,32.001043,35.970014,4.0,Coffee Shop,Restaurant,Asian Restaurant,Electronics Store,Duty-free Shop,Donut Shop,Doner Restaurant,Dog Run,Diner,Dessert Shop
4,Amman,Abu-Nsair,32.052860,35.876480,4.0,Pizza Place,Gym / Fitness Center,Shopping Mall,Food & Drink Shop,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Diner,Dessert Shop


In [80]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(amman_merged['Latitude'], amman_merged['Longitude'],
                                  amman_merged['Location'],amman_merged['Cluster Labels'].astype(int)):
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

        
map_clusters
